# Objective

The objective of this notebook is get hands-on experience on cleaning a "dirty" dataset.
Often, datasets are created from "free-text" fields. In free-text fields, data validation is not enforced and as a result, many conventions co-exist within the same column of data.
Dirty data can also ocurr when collection information from different sources. If these sources use different conventions to represent such data, additional efforts are required to homogenize it at a later stage.

## Setup

In [1]:
!pip install -q eccd_datasets pygradus

In [2]:
STUDENT_NAME = "victoria-tejera"
COURSE_NAME = "eccd-oct22"
EXERCISE_NAME = "cleaning-a-dataset"

In [3]:
import pandas as pd
import numpy as np
from eccd_datasets import load_lingerie
from pygradus import create_exercise, check_solution
import re

In [4]:
datasets = load_lingerie()

In [5]:
datasets.keys()

dict_keys(['ae_com', 'amazon_com', 'btemptd_com', 'calvinklein_com', 'hankypanky_com', 'macys_com', 'shop_nordstrom_com', 'us_topshop_com', 'victoriassecret_com'])

## The different datasets on their own

In [6]:
datasets["ae_com"].head()

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
0,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Rugged Green
1,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Natural Nude
2,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",True Black
3,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",White
4,Aerie Everyday Loves Lace Cheeky,12.50 USD,12.50 USD,https://www.ae.com/aerie-everyday-loves-lace-c...,AERIE,Cheekies,Ae US,Introducing Everyday Loves™: Made with love. E...,5.0,8.0,"[""Soft lace with the right amount of stretch"",...","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]","[""XS"", ""S"", ""M"", ""L"", ""XL"", ""XXL""]",Royal Navy


In [7]:
datasets["amazon_com"].head()

,product_name,mrp,price,pdp_url,brand_name,product_category,retailer,description,rating,review_count,style_attributes,total_sizes,available_size,color
0,Calvin Klein Women's Sheer Marquisette Demi Un...,$36.00,$32.40,https://www.amazon.com/-/dp/B01NAVD98J?th=1&psc=1,Calvin-Klein,Bras,Amazon US,"An unlined demi cup bra featuring sheer, sexy ...",4.5,47,"[ 72% Nylon, 28% Elastane , Imported , hook an...","30B , 30C , 30D , 30DD , 32A , 32B , 32C , 32D...","30B , 30C , 30D , 30DD , 32B , 32C , 32D , 32D...",Bare
1,Wacoal Embrace Lace Bikini Panty,$27.00,$27.00,https://www.amazon.com/-/dp/B0011YQFNK?th=1&psc=1,Wacoal,Panties,Amazon US,Embrace lace bikini offers great fit and match...,4.4,91,"[ 100% Nylon , Imported , Hand Wash , 1.2"" hig...","Small , Medium , Large , X-Large","Small , Medium , X-Large",Large
2,Wacoal Women's Slimline Seamless Minimizer Bra,$65.00,$65.00,https://www.amazon.com/-/dp/B000T3606Q?th=1&psc=1,Wacoal,Bras,Amazon US,Seamless underwire minimizer bra gives great s...,4.3,298,"[ Cups: 100% Polyester; Back: 71% Nylon, 29% S...","32DD , 32DDD , 34C , 34D , 34DD , 34DDD , 36C ...","32DD , 32DDD , 34C , 34D , 34DD , 34DDD , 36C ...",Black
3,Hanky Panky Womens Signature Lace Retro V-Kini,$36.00,$36.00,https://www.amazon.com/-/dp/B003Y6AX0Y?th=1&psc=1,Hanky-Panky,Panties,Amazon US,All-day comfort describes this figure-flatteri...,4.4,46,"[ Made in USA , All-day comfort describes this...","Small , Medium , Large , X-Large","Small , Medium , X-Large",Large
4,Wacoal Women's Red Carpet Strapless Bra,$65.00,$65.00,https://www.amazon.com/-/dp/B01CEBGQA0?th=1&psc=1,Wacoal,Bras,Amazon US,"Red Carpet full figure strapless fits great, s...",4.4,747,"[ 91% Nylon, 9% Spandex , Imported , hook and ...","30D , 30DD , 30DDD , 30G , 32C , 32D , 32DD , ...","30D , 30DD , 30DDD , 30G , 32C , 32D , 32DD , ...",Pecan


### Joining all the datasets into one

In [8]:
df = pd.concat(datasets.values())

In [9]:
df.shape

(613143, 14)

In [10]:
df["price"].unique()

array(['12.50 USD', '9.50 USD', '7.50 USD', ..., 'Rp474.522', 'Rp543.294',
       'Rp343.857'], dtype=object)

# Calculating statistics on the dataset

## Unifying Victoria's Secret

In [11]:
def sust(st):
  if "ict" in st:
      return "victoria's secret"
  else: return st

In [12]:
def unify_victoria_secret(df):
    """
    We want that all brands that are related to Victoria's Secret
    have `victoria's secret` as their brand instead of what they
    currently have.
    """
    df_s = df.copy()
    new_string = "victoria's secret"
    # Write your code here

    df_s["brand_name"] = df_s["brand_name"].apply(sust)
    return df_s

In [13]:
df["brand_name"]

0                     AERIE
1                     AERIE
2                     AERIE
3                     AERIE
4                     AERIE
                ...        
453381    Victoria's Secret
453382    Victoria's Secret
453383    Victoria's Secret
453384    Victoria's Secret
453385    Victoria's Secret
Name: brand_name, Length: 613143, dtype: object

In [14]:
df_unified = unify_victoria_secret(df)

In [15]:
df_unified["brand_name"]

0                     AERIE
1                     AERIE
2                     AERIE
3                     AERIE
4                     AERIE
                ...        
453381    victoria's secret
453382    victoria's secret
453383    victoria's secret
453384    victoria's secret
453385    victoria's secret
Name: brand_name, Length: 613143, dtype: object

In [16]:
answer_victoria_secret =df_unified[df_unified["brand_name"] == "victoria's secret"].shape[0]
print(answer_victoria_secret)

453453


## Cleaning up the price

In this sectino we are going to transform the `price` column into a float column in USD dolars.

For this, be careful of the different formats in the data.

For simplicity, you might assume that all the prices are in USD dolars, regarding of the symbol of the currency used.

In [17]:
df["price"]

0         12.50 USD
1         12.50 USD
2         12.50 USD
3         12.50 USD
4         12.50 USD
            ...    
453381      $48.00 
453382      $48.00 
453383      $48.00 
453384      $48.00 
453385      $48.00 
Name: price, Length: 613143, dtype: object

In [18]:
n1="35.00\n\t\t\t\t\t            \t\t\t\t\t\t\t\t            \t\t\t\t\t            \t\t\t\n\t\t\t\t\t            \t\t\n\t\t\t\t\t\t            \t\t\t\tNow\xa0'"
float(n1.split("\n")[0])


35.0

In [19]:
def clean(price):
  if "–" in price:
    return min(float(price.split("–")[0].split("$")[1].strip()),float(price.split("–")[1].split("$")[1].strip()))
  elif "-" in price:
    return min(float(price.split("-")[0].split("$")[1].strip()),float(price.split("-")[1].split("$")[1].strip()))
  elif "USD" in price:
    return float(price.strip().split()[0])
  elif "\n" in price:
    return 
  elif "$" in price:
    if "\n" in price:
      return float(price.strip().split("$")[1].split("\n")[0])
    else: return float(price.strip().split("$")[1])
  elif "Rp" in price:
    return float(price.strip().split("p")[1])

In [20]:
def ClearRangePrice(df):    
    if "\t" in df:
        return df.replace("\n","").replace("\t","").strip().split('Now\xa0')[1]
    elif len(df)>=13:
        return df.replace("–",',').replace("-",",").split(',')[0]   
    else:
        return df

In [21]:
def ClearNonDigit(df):   
    clear = re.compile(r'[^\d\.]')
    return clear.sub('', df)

In [22]:
def clean_price(df):
    """
    In this function we will transform the
    `price` column into a column of floats.
    In case a product has more than one price,
    return the lowest one.
    """
    
    df["price"] = df["price"].apply(ClearRangePrice)
    df["price"] = df["price"].apply(ClearNonDigit)
    df['price'] = df['price'].astype('float')
    return df


In [23]:
df_clean = clean_price(df_unified)

In [24]:
answer_unified_price_sum = df_clean["price"].sum()
print(answer_unified_price_sum)

20191976.02


In [25]:
def low_high_product_mean(df):
    """
    Finally, we will calculate `product_category` with the lowest and highest mean price
    for the brand Victoria's Secret.
    """
    df_prom = df_clean.groupby('product_category')['price'].mean()
    lowest_mean = df_prom.sort_values()[0]
    highest_mean = df_prom.sort_values()[-1]
    # Write your code here
    return lowest_mean,highest_mean

In [26]:
lowest_mean, highest_mean = low_high_product_mean(df_clean)
assert np.allclose(lowest_mean, 3.6203030303030)
print("highest_mean", highest_mean)

highest_mean 98.0


In [27]:

proposed_solution = {
'attempt': {
    'course_name': COURSE_NAME,
    'exercise_name': EXERCISE_NAME,
    'username': STUDENT_NAME,
},
'task_attempts': [
         {
            "name": "victoria secret",
            "answer": str(answer_victoria_secret),
         },
         {
            "name": "price unification",
            "answer": str(answer_unified_price_sum),
         },
        {
            "name": "highest mean",
            "answer": str(highest_mean),
         },
]

}
check_solution(proposed_solution)
    

|                    Task Name                     |       Status       |
|--------------------------------------------------|--------------------|
|--------------------------------------------------|--------------------|
|                 victoria secret                  |      Correct       |
|--------------------------------------------------|--------------------|
|                price unification                 |      Correct       |
|--------------------------------------------------|--------------------|
|                   highest mean                   |      Correct       |
|--------------------------------------------------|--------------------|


In [28]:
#HIIIII